In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def fetch_medicamento(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

def process_medicamento(medicamento, lista_marcas):
    try:
        nome = medicamento.find("h1", class_="name").text.strip()
    except:
        nome = 'sem'
    try: 
        preco = float(medicamento.find("p", class_="unit-price").text.replace('R$ ', '').replace(',', '.'))
    except: 
        preco = 'sem'
    try:
        ean = medicamento.find(itemprop="gtin13").get('content')
    except:
        ean = 'sem'
    try: 
        desconto = float(medicamento.find("span", class_="discount").find('b').text.replace('%', ''))
    except: 
        desconto = 'sem'
    try:
        preco_desconto = round(preco * (1-desconto/100), 2)
    except:
        preco_desconto = 'sem'
    # Procurar pela marca no HTML
    marca_html = medicamento.find("a", class_="text-primary font-weight-bold")
    if marca_html:
        marca = marca_html.text.strip()
    else:
        # Procurar pela marca no nome do medicamento
        for marca in lista_marcas:
            if marca in nome:
                break
        else:
            marca = None
    return nome, preco, preco_desconto, desconto, ean, marca



def main():
    URLS=[]
    urls_primaria = []
    for i in range(1, 229):
        # URL da página de medicamentos
        urls_primaria.append("https://www.farmaponte.com.br/medicamentos/?p="+str(i))

    with ThreadPoolExecutor(max_workers=10) as executor:
        responses_primaria = list(executor.map(fetch_medicamento, urls_primaria))
        
        for response in responses_primaria:
            # Criar objeto BeautifulSoup
            soup = BeautifulSoup(response, "html.parser")
            # Encontrar os containers dos medicamentos
            medicamentos = soup.find_all("div", class_="item-product")
            # Extrair dados para cada medicamento
            for medicamento in medicamentos:
                nome = medicamento.find("h2", class_="title").text.strip()
                try:
                    preco = medicamento.find("p", class_="pix-price").text.strip()
                except: 
                    preco='sem'
                URL = "https://www.farmaponte.com.br" + medicamento.find("a", class_='item-image').get('href')
                # Imprimir dados
                URLS.append(URL)
                
    print(len(URLS))
    print(URLS[0])

    NOME = []
    PRECO=[]
    PRECO_DESCONTO = []
    DESCONTO = []
    EAN=[]
    MARCA=[]
    lista_marcas = ["1Farma", "ADV Farma", "Abbott do Brasil", "Accord Farma", "Aché",
    "Addera", "Allergan", "Althaia", "Apsen", "Arese Pharma", "Aspen Pharma",
    "Astellas Farma", "Astrazeneca", "Baldacci", "Bausch + Lomb", "Bayer",
    "Besins", "Biolab", "Biolab Genéricos", "Biosintética - Aché", "Blanver S.A.",
    "Blau", "Boehringer Ingelheim", "Boiron", "Brasterápica", "Bravir",
    "Catarinense Pharma", "Cellera Farma", "Centrum", "Chiesi", "Cifarma",
    "Cimed", "Cosmed", "Daiichi-Sankyo", "Diffucap Chemobras", "EMS",
    "EMS Sigma Pharma", "Eli Lilly", "Eurofarma", "Exeltis", "Farmax",
    "Farmoquímica", "FiberMais", "GSK", "Galderma", "Geolab", "Germed Pharma",
    "Glenmark", "Granado Pharmácias", "Gross", "Grünenthal", "Gummy", "Hebron",
    "Herbarium", "Janssen-Cilag", "Kley Hertz Farmacêutica", "Laboratório Cristália",
    "Legrand", "Leo Pharma", "Libbs", "Marjan Farma", "Medley", "Medquímica",
    "Megalabs", "Merck", "Merck Sharp & Dohme", "Momenta Farma", "Multilab", "Mylan",
    "Myralis", "Natulab", "Neo Química", "Nova Química", "Novartis", "Novo Nordisk",
    "Organon", "P&G", "PF Consumer Healthcare", "Pfizer", "Pharlab", "Polinize Florais",
    "Prati-Donaduzzi", "Ranbaxy", "Reckitt Benckiser", "Roche", "Sandoz", "Sanofi",
    "Servier", "Supera", "Takeda", "Teuto", "Theraskin", "Top Beauty", "Torrent",
    "União Química", "Vick", "Vitamedic", "Wyeth/Pfizer", "Zambon", "Zodiac"]  # Adicione suas marcas aqui
    with ThreadPoolExecutor(max_workers=12) as executor:
        responses = list(executor.map(fetch_medicamento, URLS))
        
        for response in responses:
            soup = BeautifulSoup(response, "html.parser")
            medicamentos = soup.find_all("div", class_="content")
            for medicamento in medicamentos:
                nome, preco, preco_desconto, desconto, ean, marca = process_medicamento(medicamento, lista_marcas)
                NOME.append(nome)
                PRECO.append(preco)
                PRECO_DESCONTO.append(preco_desconto)
                DESCONTO.append(desconto)
                EAN.append(ean)
                MARCA.append(marca)

    df = pd.DataFrame()
    df['Nome'] = NOME
    df['Preço sem desconto'] = PRECO
    df['Preço com desconto'] = PRECO_DESCONTO
    df['Desconto (porcentagem)'] = DESCONTO 
    df['Código EAN'] = EAN
    df['Marca'] = MARCA
    print(df)
    df.to_csv('base_farmaponte.csv')

if __name__ == "__main__":
    main()


4560
https://www.farmaponte.com.br/dipirona-gts-10ml-ger/p
                                                   Nome Preço sem desconto  \
0                            Dipirona Gotas 10ml Germed               6.93   
1     Dipirona Monoidratada Biosintética - Aché 50mg...              16.78   
2     Citrato de Sildenafila Medley 50mg, caixa com ...              17.51   
3     Tadalafila Eurofarma 5mg, caixa com 30 comprim...             347.64   
4     Tadalafila Eurofarma 20mg, caixa com 4 comprim...             185.37   
...                                                 ...                ...   
4555  Redoxon 2g Caixa com 10 comprimidos efervescentes              26.34   
4556        Ancoron 100Mg Com 60 Comprimidos Revestidos              38.12   
4557                 Torval Cr 500Mg Com 60 Comprimidos             145.47   
4558        Noex Budesonida 64mcg Spray Nasal 120 doses              76.03   
4559  Cloridrato De Ondansetrona 8Mg Neo Quimica, Ca...             148.39   

    

In [4]:
lista_marcas = ["1Farma", "ADV Farma", "Abbott do Brasil", "Accord Farma", "Aché",
    "Addera", "Allergan", "Althaia", "Apsen", "Arese Pharma", "Aspen Pharma",
    "Astellas Farma", "Astrazeneca", "Baldacci", "Bausch + Lomb", "Bayer",
    "Besins", "Biolab", "Biolab Genéricos", "Biosintética - Aché", "Blanver S.A.",
    "Blau", "Boehringer Ingelheim", "Boiron", "Brasterápica", "Bravir",
    "Catarinense Pharma", "Cellera Farma", "Centrum", "Chiesi", "Cifarma",
    "Cimed", "Cosmed", "Daiichi-Sankyo", "Diffucap Chemobras", "EMS",
    "EMS Sigma Pharma", "Eli Lilly", "Eurofarma", "Exeltis", "Farmax",
    "Farmoquímica", "FiberMais", "GSK", "Galderma", "Geolab", "Germed Pharma",
    "Glenmark", "Granado Pharmácias", "Gross", "Grünenthal", "Gummy", "Hebron",
    "Herbarium", "Janssen-Cilag", "Kley Hertz Farmacêutica", "Laboratório Cristália",
    "Legrand", "Leo Pharma", "Libbs", "Marjan Farma", "Medley", "Medquímica",
    "Megalabs", "Merck", "Merck Sharp & Dohme", "Momenta Farma", "Multilab", "Mylan",
    "Myralis", "Natulab", "Neo Química", "Nova Química", "Novartis", "Novo Nordisk",
    "Organon", "P&G", "PF Consumer Healthcare", "Pfizer", "Pharlab", "Polinize Florais",
    "Prati-Donaduzzi", "Ranbaxy", "Reckitt Benckiser", "Roche", "Sandoz", "Sanofi",
    "Servier", "Supera", "Takeda", "Teuto", "Theraskin", "Top Beauty", "Torrent",
    "União Química", "Vick", "Vitamedic", "Wyeth/Pfizer", "Zambon", "Zodiac"]

In [6]:
# Carregar o arquivo CSV como um DataFrame
df_farmaponte = pd.read_csv('base_farmaponte.csv')

# Exibir as primeiras linhas do DataFrame para verificar se foi carregado corretamente
df_farmaponte.head()

,Unnamed: 0,Nome,Preço sem desconto,Preço com desconto,Desconto (porcentagem),Código EAN,Marca
0,0,Dipirona Gotas 10ml Germed,6.93,2.08,70.0,7896004719115,NaN
1,1,Dipirona Monoidratada Biosintética - Aché 50mg...,16.78,16.78,0.0,7896181911036,Biosintética - Aché
2,2,"Citrato de Sildenafila Medley 50mg, caixa com ...",17.51,14.01,20.0,7896422525190,Medley
3,3,"Tadalafila Eurofarma 5mg, caixa com 30 comprim...",347.64,27.81,92.0,7891317127725,Eurofarma
4,4,"Tadalafila Eurofarma 20mg, caixa com 4 comprim...",185.37,14.83,92.0,7891317127800,Eurofarma


In [7]:
def preencher_marca(df, lista_marcas):
    for i, nome in enumerate(df['Nome']):
        if pd.isna(df.at[i, 'Marca']):
            for marca in lista_marcas:
                if any(word.lower() in nome.lower() for word in marca.split()):
                    df.at[i, 'Marca'] = marca
                    break

In [8]:
dataframe = df_farmaponte.copy()

In [9]:
preencher_marca(dataframe, lista_marcas)

In [10]:
dataframe.head(50)

,Unnamed: 0,Nome,Preço sem desconto,Preço com desconto,Desconto (porcentagem),Código EAN,Marca
0,0,Dipirona Gotas 10ml Germed,6.93,2.08,70.0,7896004719115,Germed Pharma
1,1,Dipirona Monoidratada Biosintética - Aché 50mg...,16.78,16.78,0.0,7896181911036,Biosintética - Aché
2,2,"Citrato de Sildenafila Medley 50mg, caixa com ...",17.51,14.01,20.0,7896422525190,Medley
3,3,"Tadalafila Eurofarma 5mg, caixa com 30 comprim...",347.64,27.81,92.0,7891317127725,Eurofarma
4,4,"Tadalafila Eurofarma 20mg, caixa com 4 comprim...",185.37,14.83,92.0,7891317127800,Eurofarma
5,5,"Apevitin BC 4mg, caixa com 1 frasco com 240mL ...",50.77,20.82,59.0,7896004700502,EMS
6,6,"Citrato de Sildenafila Eurofarma 50mg, caixa c...",30.62,13.78,55.0,7891317477592,Eurofarma
7,7,Materna Nestle 30 Capsula,89.9,76.42,15.0,7891045031257,Wyeth/Pfizer
8,8,Leite de Magnésia Eno frasco com 350mL de susp...,26.03,23.43,10.0,7896015591946,GSK
9,9,Odorizante De Ambiente Bendita Canfora 8 Tabletes,15.72,14.15,10.0,7896186002548,Bravir


In [11]:
valores_nan = dataframe['Marca'].isna().sum()

print("Número de valores NaN na coluna 'Nome':", valores_nan)

Número de valores NaN na coluna 'Nome': 311


In [12]:
dataframe.columns

Index(['Unnamed: 0', 'Nome', 'Preço sem desconto', 'Preço com desconto',
       'Desconto (porcentagem)', 'Código EAN', 'Marca'],
      dtype='object')

In [13]:
dataframe = dataframe.drop(columns=['Unnamed: 0'])

In [14]:
dataframe.head()

,Nome,Preço sem desconto,Preço com desconto,Desconto (porcentagem),Código EAN,Marca
0,Dipirona Gotas 10ml Germed,6.93,2.08,70.0,7896004719115,Germed Pharma
1,Dipirona Monoidratada Biosintética - Aché 50mg...,16.78,16.78,0.0,7896181911036,Biosintética - Aché
2,"Citrato de Sildenafila Medley 50mg, caixa com ...",17.51,14.01,20.0,7896422525190,Medley
3,"Tadalafila Eurofarma 5mg, caixa com 30 comprim...",347.64,27.81,92.0,7891317127725,Eurofarma
4,"Tadalafila Eurofarma 20mg, caixa com 4 comprim...",185.37,14.83,92.0,7891317127800,Eurofarma


In [17]:
dataframe.to_excel('farmaponte_excel.xlsx', index = False)